In [12]:
import getpass
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["LANGSMITH_PROJECT"] = "data-txt-test"

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

model = "gpt-3.5-turbo"

llm = ChatOpenAI(temperature=0.5, model=model, max_tokens=4096)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [24]:
from langchain_chroma import Chroma
import chromadb
chroma_client = chromadb.HttpClient(host='localhost', port=8000)
print(chroma_client.heartbeat())

vector_store = Chroma(
    collection_name="real-state",
    embedding_function=embeddings,
    client=chroma_client
)

1739318358054658000


In [29]:
results = vector_store.similarity_search_by_vector(
    embedding=embeddings.embed_query("proyecto a"), k=1
)
print(results)

[]


## Retriever

In [30]:
retriever = vector_store.as_retriever()


## Prompt template

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

template = """Responde las siguientes preguntas con base en los siguientes contexto:
Eres un asistente virtual especializado en una agencia inmobiliaria. Tu tarea es proporcionar información precisa y útil sobre los proyectos inmobiliarios disponibles. Utiliza únicamente la información proporcionada en los documentos para responder.

Proyectos disponibles:
1. **Proyecto A**: Residencial "Bosque Verde"
2. **Proyecto B**: Comercial "Plaza Sol"
3. **Proyecto C**: Residencial de lujo "Altos del Mar"

Instrucciones:
- Responde de manera clara y concisa.
- Si no tienes información suficiente, indica que no cuentas con los detalles necesarios.
- Prioriza la información sobre ubicación, precios, características y disponibilidad.
- El año actual es 2025.

{context}
"""
prompt_template = ChatPromptTemplate([
    ("system", template),
    ("user", "{question}")
])

## RAG Chain

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [17]:
response = rag_chain.invoke("dame precios?")
print(response)

Claro, ¿sobre cuál proyecto te gustaría saber los precios?


## RAG Graph

In [18]:
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"],k=3)
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt_template.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [19]:
response = graph.invoke({"question": "estoy interesado en comprar un apartamento y tengo un presupuesto de 1.200.000 usd y a la misma vez, estoy pensando en algun local comercial, que podria hacer?, dame precios y especificaciones de los inmuebles"})
print(response["answer"])

Para tu presupuesto de 1.200.000 USD, te recomendaría considerar el **Proyecto C: Residencial de lujo "Altos del Mar"**. En este proyecto, actualmente hay disponibles apartamentos de 3 habitaciones con vista al mar, amplias terrazas, acabados de lujo y servicios exclusivos como gimnasio, piscina y seguridad las 24 horas. Los precios de los apartamentos en este proyecto oscilan entre 900.000 USD y 1.500.000 USD, dependiendo de la ubicación y tamaño del apartamento.

En cuanto a la opción de un local comercial, te recomendaría considerar el **Proyecto B: Comercial "Plaza Sol"**. En este proyecto, hay locales comerciales disponibles con excelente ubicación en una zona de alto tráfico, perfectos para negocios como restaurantes, tiendas o servicios. Los precios de los locales comerciales en este proyecto varían entre 800.000 USD y 1.200.000 USD, dependiendo del tamaño y la ubicación del local.

Si deseas más detalles sobre algún proyecto en particular, por favor házmelo saber.


In [20]:
retrieved_docs = vector_store.similarity_search("estoy interesado en comprar un apartamento y tengo un presupuesto de 1.200.000 usd y a la misma vez, estoy pensando en algun local comercial, que podria hacer?, dame precios y especificaciones de los inmuebles",k=3)
print(retrieved_docs)

[]
